## Installing and loading the Dataset


In [ ]:
!pip3 install -q datasets

In [ ]:
from datasets import load_dataset

dataset = load_dataset("surrey-nlp/PLOD-CW")

Installing Necessary libariries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

Create 3 different Dataframe for further Analysis

In [ ]:
df = pd.DataFrame(dataset['train'])

In [ ]:
df_test = pd.DataFrame(dataset['test'])

In [ ]:
df_valid =pd.DataFrame(dataset['validation'])

Converting List to Strings using Join function

In [ ]:
df['text_for_tfidf'] = df['tokens'].apply(lambda x: ' '.join(x))
df_test['text_for_tfidf'] = df_test['tokens'].apply(lambda x: ' '.join(x))
df_valid['text_for_tfidf'] = df_valid['tokens'].apply(lambda x: ' '.join(x))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000)  # Adjust max_features as necessary

# Vectorize the text data
X_train = tfidf_vectorizer.fit_transform(df['text_for_tfidf'])
y_train = df['ner_tags']

In [ ]:
X_test = tfidf_vectorizer.transform(df_test['text_for_tfidf'])
X_valid = tfidf_vectorizer.transform(df_valid['text_for_tfidf'])


In [ ]:
print(X_train[:10])

  (0, 1511)	0.35337378350180654
  (0, 1875)	0.17526881289553137
  (0, 2922)	0.4157488441830654
  (0, 3361)	0.47556360652815105
  (0, 3892)	0.3773785775769426
  (0, 4345)	0.09952399408663466
  (0, 4361)	0.2398456304763198
  (0, 4821)	0.18403251985445085
  (0, 4942)	0.44909515253527005
  (1, 532)	0.04967714296321862
  (1, 1227)	0.13634789692625202
  (1, 1254)	0.3955763810768054
  (1, 1304)	0.16482472055295558
  (1, 1873)	0.15437390238837814
  (1, 1988)	0.20641998854251215
  (1, 1999)	0.4128399770850243
  (1, 2518)	0.15103661686701095
  (1, 2616)	0.20641998854251215
  (1, 2696)	0.19109284784829908
  (1, 2950)	0.20641998854251215
  (1, 3023)	0.21858582453318098
  (1, 3463)	0.2654130153718114
  (1, 4345)	0.04574474162789979
  (1, 4452)	0.21858582453318098
  (1, 4491)	0.21858582453318098
  :	:
  (8, 2077)	0.26469742168928406
  (8, 2812)	0.26469742168928406
  (8, 3281)	0.1449093915325811
  (8, 3507)	0.2116225367130834
  (8, 4019)	0.25573712965884293
  (8, 4245)	0.24222614319965335
  (8, 4344)

In [ ]:
y_train = df['ner_tags']
y_test = df_test['ner_tags']
y_valid = df_valid['ner_tags']

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
y_train_binarized = mlb.fit_transform(y_train)
y_test_binarized = mlb.transform(y_test)
y_valid_binarized = mlb.transform(y_valid)

In [ ]:
print(y_train_binarized[:10])

[[1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 0 1 0]
 [1 0 1 0]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]]


**System 1 - No preprocessing , Tfidf , SVM(Hyperparameter Optimization)**

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler


model = OneVsRestClassifier(SVC(kernel='linear'))


param_grid = {
    "estimator__C": [1],
    "estimator__kernel": ['rbf'],
    "estimator__gamma": ['scale']
}


grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')


grid_search.fit(X_train, y_train_binarized)





/user/HS401/pb00885/.local/lib/python3.10/site-packages/sklearn/multiclass.py:90: UserWarning: Label 2 is present in all training examples.
  warnings.warn(
/user/HS401/pb00885/.local/lib/python3.10/site-packages/sklearn/multiclass.py:90: UserWarning: Label 2 is present in all training examples.
  warnings.warn(
/user/HS401/pb00885/.local/lib/python3.10/site-packages/sklearn/multiclass.py:90: UserWarning: Label 2 is present in all training examples.
  warnings.warn(
/user/HS401/pb00885/.local/lib/python3.10/site-packages/sklearn/multiclass.py:90: UserWarning: Label 2 is present in all training examples.
  warnings.warn(
/user/HS401/pb00885/.local/lib/python3.10/site-packages/sklearn/multiclass.py:90: UserWarning: Label 2 is present in all training examples.
  warnings.warn(
/user/HS401/pb00885/.local/lib/python3.10/site-packages/sklearn/multiclass.py:90: UserWarning: Label 2 is present in all training examples.
  warnings.warn(


GridSearchCV(cv=5,
             estimator=OneVsRestClassifier(estimator=SVC(kernel='linear')),
             param_grid={'estimator__C': [1], 'estimator__gamma': ['scale'],
                         'estimator__kernel': ['rbf']},
             scoring='accuracy')

In [ ]:
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))


Best parameters: {'estimator__C': 1, 'estimator__gamma': 'scale', 'estimator__kernel': 'rbf'}
Best cross-validation score: 0.75


In [ ]:
y_pred1 = model_best.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, hamming_loss
print("Accuracy:", accuracy_score(y_test_binarized, y_pred1))
print("Precision:", precision_score(y_test_binarized, y_pred1, average='samples'))
print("Recall:", recall_score(y_test_binarized, y_pred1, average='samples'))
print("F1 Score:", f1_score(y_test_binarized, y_pred1, average='samples'))
print("Hamming Loss:", hamming_loss(y_test_binarized, y_pred1))

Accuracy: 0.6797385620915033
Precision: 0.8349673202614379
Recall: 0.9967320261437909
F1 Score: 0.8780578898225958
Hamming Loss: 0.16666666666666666


**System 2 - No preprocessing , Tfidf , KNN(Hyperparameter Optimization)**

In [ ]:
df['Tokens'] = df['tokens'].apply(lambda x: ' '.join(eval(x)) if isinstance(x, str) else ' '.join(x))
df['NER'] = df['ner_tags'].apply(lambda x: eval(x) if isinstance(x, str) else x)

In [ ]:
df_test['Tokens'] = df_test['tokens'].apply(lambda x: ' '.join(eval(x)) if isinstance(x, str) else ' '.join(x))
df_test['NER'] = df_test['ner_tags'].apply(lambda x: eval(x) if isinstance(x, str) else x)

In [ ]:
df_valid['Tokens'] = df_valid['tokens'].apply(lambda x: ' '.join(eval(x)) if isinstance(x, str) else ' '.join(x))
df_valid['NER'] = df_valid['ner_tags'].apply(lambda x: eval(x) if isinstance(x, str) else x)

In [ ]:
df['tokens_knn'] = df['Tokens']
df_test['tokens_knn'] = df_test['Tokens']
df_valid['tokens_knn'] = df_valid['Tokens']

In [ ]:
tfidf = TfidfVectorizer(max_features=5000)

In [ ]:
X_train = tfidf.fit_transform(df['tokens_knn'])
X_test = tfidf.transform(df_test['tokens_knn'])
X_valid = tfidf.transform(df_valid['tokens_knn'])

In [ ]:
y_train = df['ner_tags']
y_test = df_test['ner_tags']
y_valid = df_valid['ner_tags']

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
y_train_binarized = mlb.fit_transform(y_train)
y_test_binarized = mlb.transform(y_test)
y_valid_binarized = mlb.transform(y_valid)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

# Initialize the model
knn = KNeighborsClassifier()


param_grid1 = {
    'n_neighbors': [51],
    'weights': ['distance'],
    'metric': ['euclidean']
}


grid_search1 = GridSearchCV(knn, param_grid1, cv=5, verbose=1, scoring='accuracy')

# Fit the model to the data
model5 =grid_search1.fit(X_train, y_train_binarized)


print("Best parameters:", grid_search1.best_params_)
print("Best cross-validation score: {:.2f}".format(grid_search1.best_score_))



Fitting 5 folds for each of 1 candidates, totalling 5 fits
Best parameters: {'metric': 'euclidean', 'n_neighbors': 51, 'weights': 'distance'}
Best cross-validation score: 0.75


In [ ]:
y_pred5 = model5.predict(X_test)
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, hamming_loss
print("Accuracy:", accuracy_score(y_test_binarized, y_pred5))
print("Precision:", precision_score(y_test_binarized, y_pred5, average='samples'))
print("Recall:", recall_score(y_test_binarized, y_pred5, average='samples'))
print("F1 Score:", f1_score(y_test_binarized, y_pred5, average='samples'))
print("Hamming Loss:", hamming_loss(y_test_binarized, y_pred5))

Accuracy: 0.673202614379085
Precision: 0.8306100217864923
Recall: 0.9967320261437909
F1 Score: 0.8747899159663867
Hamming Loss: 0.16993464052287582
